In [2]:
from dotenv import load_dotenv
from sqlalchemy.engine import URL, create_engine
import os

In [3]:
load_dotenv(override=True)  # take environment variables from .env.

True

In [4]:
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": os.getenv('conn_str')})

In [5]:
%load_ext sql

In [6]:
%env  DATABASE_URL=$connection_url

env: DATABASE_URL=mssql+pyodbc://?odbc_connect=DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BSERVER%3Djfpdb.database.windows.net%3BDATABASE%3Djfpdb%3BUID%3Djfp%3BPWD%3D16%2AraccIEjos


In [24]:
%%sql
WITH gender_total_disease AS (
    SELECT 
        gender,
        count(*) total_disease
    FROM healthcare.health_data 
    WHERE heart_disease = 1  
    GROUP BY gender
),
gender_total_count AS (
    SELECT
        gender,
        count(*) total_count
    FROM healthcare.health_data
    GROUP BY gender
)
SELECT
    a.gender,
    CAST((a.total_disease*1.0 / b.total_count) * 100 AS decimal(4,2)) as gender_disease_pct
FROM
    gender_total_disease a
JOIN
    gender_total_count b ON a.gender = b.gender;


 * mssql+pyodbc://?odbc_connect=DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BSERVER%3Djfpdb.database.windows.net%3BDATABASE%3Djfpdb%3BUID%3Djfp%3BPWD%3D16%2AraccIEjos
Done.


gender,gender_disease_pct
Male,50.45
Female,50.24


In [34]:
%%sql 
WITH age_grouping AS (
    SELECT
        *,
        CASE
            WHEN age < 30 then 'under 30'
            WHEN age < 41 then '30 - 40'
            WHEN age < 51 then '40 - 50'
            WHEN age < 61 then '50 - 60'
            WHEN age < 71 then '60 - 70'
            else 'over 70'
        END as AGEGROUP
    FROM healthcare.health_data
)
SELECT
    AGEGROUP,
    COUNT(*) patients_with_disease
FROM
    age_grouping
WHERE
    heart_disease = 1
GROUP BY
    AGEGROUP

 * mssql+pyodbc://?odbc_connect=DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BSERVER%3Djfpdb.database.windows.net%3BDATABASE%3Djfpdb%3BUID%3Djfp%3BPWD%3D16%2AraccIEjos
Done.


AGEGROUP,patients_with_disease
50 - 60,3640
under 30,3521
60 - 70,3552
30 - 40,3937
40 - 50,3568
over 70,7089


In [23]:
%%sql
SELECT
    TOP 1 '40 - 50' agegroup, resting_ecg_results
FROM
    healthcare.health_data
WHERE
    heart_disease = 1 AND age BETWEEN 40 AND 50
GROUP BY
    resting_ecg_results
ORDER BY COUNT(*) DESC;

 * mssql+pyodbc://?odbc_connect=DRIVER%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BSERVER%3Djfpdb.database.windows.net%3BDATABASE%3Djfpdb%3BUID%3Djfp%3BPWD%3D16%2AraccIEjos
Done.


agegroup,resting_ecg_results
40 - 50,2
